In [ ]:
# Created on Jan 2021
# author: 임일

import numpy as np
import pandas as pd
# Read rating data
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('u.data', names=r_cols,  sep='\t',encoding='latin-1')
ratings = ratings.drop('timestamp', axis=1)

In [ ]:
# Rating 데이터를 test, train으로 나누고 train을 full matrix로 변환
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y, random_state=12)
rating_matrix = x_train.pivot(values='rating', index='user_id', columns='movie_id')

In [ ]:
# RMSE 계산을 위한 함수
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

def score(model):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

In [ ]:
# Train data의 모든 아이템 pair의 Cosine similarities 계산
from sklearn.metrics.pairwise import cosine_similarity
rating_matrix_t = np.transpose(rating_matrix)
matrix_dummy = rating_matrix_t.copy().fillna(0)
item_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
item_similarity = pd.DataFrame(item_similarity, index=rating_matrix_t.index, columns=rating_matrix_t.index)

def cf_IBCF(user_id, movie_id):
    if user_id in rating_matrix_t:                  # 사용자가 train set에 있는지 확인
        if movie_id in item_similarity:             # 현재 영화가 train set에 있는지 확인
            # 현재 영화와 다른 영화의 similarity 값 가져오기
            sim_scores = item_similarity[movie_id]
            # 현 사용자의 모든 rating 값 가져오기
            user_rating = rating_matrix_t[user_id]
            # 사용자가 평가하지 않은 영화 index 가져오기
            non_rating_idx = user_rating[user_rating.isnull()].index
            # 사용자가 평가하지 않은 영화 제거
            user_rating = user_rating.dropna()
            # 사용자가 평가하지 않은 영화의 similarity 값 제거
            sim_scores = sim_scores.drop(non_rating_idx)
            if len(user_rating) >= 2:
                # 현 영화에 대한 사용자의 예상 rating 계산, 가중치는 현 영화와 사용자가 평가한 영화의 유사도
                prediction = np.dot(sim_scores, user_rating) / sim_scores.sum()
            else:
                prediciton = 3.0
        else:
            prediction = 3.0
    else:
        prediction = 3.0
    return prediction

In [ ]:
# 정확도 계산
score(cf_IBCF)

In [ ]:
# 추천을 위한 데이터 읽기 (추천을 위해서는 전체 데이터를 읽어야 함)
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('u.data', names=r_cols,  sep='\t',encoding='latin-1')
ratings = ratings.drop('timestamp', axis=1)
rating_matrix = ratings.pivot(values='rating', index='user_id', columns='movie_id')

# 영화 제목 가져오기
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 
          'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 
          'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 
          'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv('u.item', sep='|', names=i_cols, encoding='latin-1')
movies = movies[['movie_id', 'title']]
movies = movies.set_index('movie_id')

# Cosine similarity 계산 (Item기준으로)
from sklearn.metrics.pairwise import cosine_similarity
rating_matrix_t = np.transpose(rating_matrix)
matrix_dummy = rating_matrix_t.copy().fillna(0)
item_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
item_similarity = pd.DataFrame(item_similarity, index=rating_matrix_t.index, columns=rating_matrix_t.index)

In [ ]:
# 추천하기
def recommender(user, n_items=10):
    # 현재 사용자의 모든 아이템에 대한 예상 평점 계산
    predictions = []
    item_ids = []
    not_rated = rating_matrix.loc[user].isnull()                        # 평가 안한 영화 확인
    for item in rating_matrix:
        if not_rated.loc[item]:                                         # 평가 안한 영화만 예측치 계산
            predictions.append(cf_IBCF(user, item))
            item_ids.append(item)
    predictions = np.array(predictions)                                 # Numpy array로 변환
    item_ids = np.array(item_ids)
    recommendations = np.argsort(predictions)[::-1][:n_items]           # 예상평점이 가장 높은 영화 선택
    recommended_items = movies.loc[item_ids[recommendations]]['title']  # 예상평점이 가장 높은 영화 제목 가져오기
    return recommended_items

In [ ]:
# 영화 추천 함수 부르기
recommender(2)